# 🎨 MangaGen - AI Manga Generation Pipeline

Generate complete manga pages with **consistent characters** from text prompts!

## 📂 Drive Structure (Auto-Created)
```
MyDrive/MangaGen/
├── Models/        # Cached SDXL (~10GB, one-time)
├── Projects/      # Your manga projects
└── CharacterLibrary/  # Reusable refs
```

## Setup
1. **Enable GPU**: Runtime → Change runtime type → T4 GPU
2. **Add API Key**: Click 🔑 → Add `GEMINI_API_KEY`
3. **Run All Cells**: Runtime → Run all

---

In [ ]:
#@title 🔧 Cell 1: Setup & Mount Drive

import os
import sys

# ALWAYS start from /content to avoid directory issues
os.chdir('/content')

# ============================================
# Mount Google Drive
# ============================================
print("📁 Mounting Google Drive...")
from google.colab import drive
drive.mount('/content/drive')

# ============================================
# Create Organized Folder Structure
# ============================================
DRIVE_BASE = '/content/drive/MyDrive/MangaGen'
FOLDERS = {
    'models': f'{DRIVE_BASE}/Models',
    'projects': f'{DRIVE_BASE}/Projects',
    'characters': f'{DRIVE_BASE}/CharacterLibrary',
}

print("📂 Creating folder structure...")
for name, path in FOLDERS.items():
    os.makedirs(path, exist_ok=True)
    print(f"   ✅ {path}")

# Set cache directories
os.environ['HF_HOME'] = FOLDERS['models']
os.environ['TRANSFORMERS_CACHE'] = FOLDERS['models']
os.environ['DIFFUSERS_CACHE'] = FOLDERS['models']

# ============================================
# Check GPU
# ============================================
import torch
print("")
if torch.cuda.is_available():
    gpu_name = torch.cuda.get_device_name(0)
    gpu_mem = torch.cuda.get_device_properties(0).total_memory / 1024**3
    print(f"✅ GPU: {gpu_name} ({gpu_mem:.1f} GB)")
else:
    print("❌ No GPU! Runtime → Change runtime type → T4 GPU")

# ============================================
# Clone Repository
# ============================================
print("")
print("📦 Cloning repository...")

# Make sure we're in /content before removing
os.chdir('/content')

# Remove old clone if exists
import shutil
if os.path.exists('/content/manga-gen'):
    shutil.rmtree('/content/manga-gen')

# Clone fresh
!git clone --branch mvp/kaggle-flux https://github.com/Barun-2005/manga-gen-ai-pipeline.git /content/manga-gen --quiet

# Verify clone succeeded
if os.path.exists('/content/manga-gen/scripts'):
    os.chdir('/content/manga-gen')
    print("✅ Repository cloned successfully")
    print(f"   Location: {os.getcwd()}")
else:
    print("❌ Clone failed! Check internet connection.")

print("")
print("🎉 Setup complete!")

In [ ]:
#@title 📚 Cell 2: Install Dependencies

import os
os.chdir('/content/manga-gen')

print("📦 Installing dependencies...")

!pip install -q diffusers==0.30.3 transformers accelerate safetensors \
    google-generativeai pydantic opencv-python-headless Pillow \
    reportlab python-dotenv tqdm xformers

print("")
print("🔍 Verifying...")
import diffusers, transformers, torch
print(f"✅ diffusers: {diffusers.__version__}")
print(f"✅ transformers: {transformers.__version__}")
print(f"✅ torch: {torch.__version__}")

from diffusers import StableDiffusionXLPipeline
print("✅ SDXL Pipeline: ready!")
print("")
print("🎉 Dependencies installed!")

In [ ]:
#@title 🔑 Cell 3: API Key Setup

import os

try:
    from google.colab import userdata
    api_key = userdata.get('GEMINI_API_KEY')
    if api_key:
        os.environ['GEMINI_API_KEY'] = api_key
        print(f"✅ API key loaded: {api_key[:8]}...")
    else:
        raise ValueError("No key")
except:
    from getpass import getpass
    api_key = getpass("🔑 Enter Gemini API key: ")
    os.environ['GEMINI_API_KEY'] = api_key
    print("✅ API key set")

In [ ]:
#@title ⚙️ Cell 4: Configuration

#@markdown ### 📝 Story Prompt
STORY_PROMPT = "Astra, a space scavenger with silver hair and orange jumpsuit, explores a derelict spaceship and discovers a glowing artifact." #@param {type:"string"}

#@markdown ### 🎨 Visual Style
STYLE = "bw_manga" #@param ["bw_manga", "color_anime"]
LAYOUT = "2x2" #@param ["2x2", "vertical_webtoon", "3_panel", "single"]

#@markdown ### ⚡ Quality Settings
INFERENCE_STEPS = 25 #@param {type:"slider", min:15, max:50, step:5}
GUIDANCE_SCALE = 7.5 #@param {type:"slider", min:5, max:12, step:0.5}

#@markdown ### 📁 Project Name
PROJECT_NAME = "my_manga" #@param {type:"string"}

import os
PROJECT_DIR = f"/content/drive/MyDrive/MangaGen/Projects/{PROJECT_NAME}"
os.makedirs(f"{PROJECT_DIR}/panels", exist_ok=True)
os.makedirs(f"{PROJECT_DIR}/output", exist_ok=True)

print(f"📋 Project: {PROJECT_NAME}")
print(f"   Style: {STYLE}, Layout: {LAYOUT}")
print(f"📂 Folder: {PROJECT_DIR}")

In [ ]:
#@title 📝 Cell 5: Generate Scene Plan (Gemini)

import os
import subprocess

os.chdir('/content/manga-gen')

# Run scene generator
cmd = f'python scripts/generate_scene_json.py "{STORY_PROMPT}" --style {STYLE} --layout {LAYOUT} --output scene_plan.json'
result = subprocess.run(cmd, shell=True, capture_output=False)

# Copy to project folder
import shutil
if os.path.exists('scene_plan.json'):
    shutil.copy('scene_plan.json', f'{PROJECT_DIR}/scene_plan.json')
    
    import json
    with open('scene_plan.json', 'r') as f:
        scene = json.load(f)
    print(f"\n✅ Scene: {scene.get('title')}")
    print(f"   Panels: {len(scene.get('panels', []))}")
else:
    print("❌ Scene generation failed. Check API key.")

In [ ]:
#@title 🎨 Cell 6: Generate Panel Images (SDXL)
#@markdown ⏱️ First run: ~5-8 min | Cached: ~3-5 min

import os
import time
import subprocess

os.chdir('/content/manga-gen')
os.makedirs('outputs', exist_ok=True)

print("🎨 Generating panels with SDXL...")
print(f"   Cache: {os.environ.get('HF_HOME')}")
print("")

start = time.time()

cmd = f'python scripts/generate_panels.py --scene scene_plan.json --output outputs/ --steps {INFERENCE_STEPS} --guidance {GUIDANCE_SCALE}'
result = subprocess.run(cmd, shell=True)

elapsed = time.time() - start
print(f"\n⏱️ Time: {elapsed/60:.1f} minutes")

# Copy to Drive
import shutil
import glob
for f in glob.glob('outputs/*.png'):
    shutil.copy(f, f"{PROJECT_DIR}/panels/")

if elapsed > 60:
    print("✅ Real images generated!")
else:
    print("⚠️ Fast - check for errors above")

In [ ]:
#@title 🖼️ Cell 7: Display Panels

from IPython.display import display, Image as IPImage
import glob, os

os.chdir('/content/manga-gen')
panels = sorted([p for p in glob.glob('outputs/panel_*.png') if 'bubbles' not in p])

if panels:
    print("🖼️ Panels:")
    for p in panels:
        print(f"\n{os.path.basename(p)}")
        display(IPImage(filename=p, width=400))
else:
    print("❌ No panels. Check Cell 6.")

In [ ]:
#@title 💬 Cell 8: Bubbles + Final Page

import os, subprocess, shutil
os.chdir('/content/manga-gen')

print("💬 Placing bubbles...")
subprocess.run('python scripts/place_bubbles.py --panels outputs/ --scene scene_plan.json --output bubbles.json', shell=True)

print("\n📄 Composing page...")
subprocess.run('python scripts/compose_page.py --panels outputs/ --bubbles bubbles.json --scene scene_plan.json --output outputs/', shell=True)

# Copy to Drive
for f in ['outputs/manga_page.png', 'outputs/manga_page.pdf']:
    if os.path.exists(f):
        shutil.copy(f, f"{PROJECT_DIR}/output/")
if os.path.exists('manga_output.zip'):
    shutil.copy('manga_output.zip', f"{PROJECT_DIR}/output/")

# Display
if os.path.exists('outputs/manga_page.png'):
    from IPython.display import display, Image as IPImage
    print("\n🎨 Final Page:")
    display(IPImage(filename='outputs/manga_page.png', width=600))
    print(f"\n📂 Saved: {PROJECT_DIR}/output/")

In [ ]:
#@title 📥 Cell 9: Download

import os
from google.colab import files

os.chdir('/content/manga-gen')

print("🎉 MANGA READY!")
print(f"📂 Drive: {PROJECT_DIR}/output/")
print("\n📥 Downloading...")

if os.path.exists('manga_output.zip'):
    files.download('manga_output.zip')
elif os.path.exists('outputs/manga_page.pdf'):
    files.download('outputs/manga_page.pdf')
else:
    print("Check Drive folder for files.")